# Exoplanet Radial Velocity GIFs
This notebook walks through:

1. Fetching confirmed exoplanet data from NASA’s Exoplanet Archive.
2. Modeling Keplerian orbits and stellar radial velocity signals.
3. Creating an animated GIF visualizing planet orbits and RV curve.

*Dependencies:* numpy, matplotlib, mplcyberpunk, imageio, requests, pandas, warnings

# 1. Imports and Style

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import mplcyberpunk
import imageio
import requests
import pandas as pd
import warnings
from io import StringIO

plt.style.use('cyberpunk')  # dark theme


## 2. Define Utility Functions

Define functions for radial velocity, semi-major axis, and ellipse coordinates with input checks

In [43]:
def radvelocity(t, P, K, e, omega, T0):
    """
    Compute radial velocity time series for a Keplerian orbit.
    """
    t = np.asarray(t, float)
    if P <= 0:
        warnings.warn('P must be >0; defaulting to 1')
        P = 1.0
    if K < 0:
        warnings.warn('K should be >=0; taking abs')
        K = abs(K)
    if not (0 <= e < 1):
        warnings.warn('e must be in [0,1); setting e=0')
        e = 0.0

    M = 2*np.pi*((t - T0) % P)/P
    E = M.copy()
    for _ in range(50):
        delta = (M + e*np.sin(E) - E)/(1 - e*np.cos(E))
        E += delta
        if np.all(np.abs(delta)<1e-8):
            break
    nu = 2*np.arctan2(np.sqrt(1+e)*np.sin(E/2), np.sqrt(1-e)*np.cos(E/2))
    return K*(np.cos(nu+omega) + e*np.cos(omega))
    

def semi_major_axis(P, M_star):
    """
    Return semi-major axis in AU.
    """
    if P <= 0:
        raise ValueError('P must be positive')
    G = 6.67408e-11
    M = M_star*1.989e30
    P_s = P*86400
    a = (G*M*P_s**2/(4*np.pi**2))**(1/3)
    return a/1.496e11


def ellipse(a, e, n=500):
    if a<=0:
        raise ValueError('a must be >0')
    if not (0<= e <1):
        warnings.warn('e must be in [0,1); setting e=0')
        e=0.0
    theta = np.linspace(0,2*np.pi,n)
    r = a*(1-e**2)/(1+e*np.cos(theta))
    return r*np.cos(theta), r*np.sin(theta)


## 3. Fetch Exoplanet Data

Use TAP service; disable low_memory to avoid warnings.


In [44]:
url = 'https://exoplanetarchive.ipac.caltech.edu/TAP/sync'
query = 'SELECT * FROM ps WHERE default_flag=1'
resp = requests.get(url, params={'query':query,'format':'csv'})
resp.raise_for_status()
planets_df = pd.read_csv(StringIO(resp.text), low_memory=False)
planets_df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,gaia_id,default_flag,pl_refname,sy_refname,...,sy_jmagerr1,sy_jmagerr2,sy_jmagstr,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_hmagstr,sy_kmag,sy_kmagerr1,sy_kmagerr2
0,Kepler-6 b,b,Kepler-6,NaN,NaN,TIC 27916356,Gaia DR2 2086636884980514304,1,<a refstr=ESTEVES_ET_AL__2015 href=https://ui....,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,0.021,-0.021,12.001&plusmn;0.021,11.706,0.019,-0.019,11.706&plusmn;0.019,11.634,0.019,-0.019
1,Kepler-491 b,b,Kepler-491,NaN,NaN,TIC 158388163,Gaia DR2 2102468134431912064,1,<a refstr=MORTON_ET_AL__2016 href=https://ui.a...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,0.021,-0.021,12.746&plusmn;0.021,12.426,0.020,-0.020,12.426&plusmn;0.020,12.354,0.019,-0.019
2,Kepler-257 b,b,Kepler-257,NaN,NaN,TIC 273132699,Gaia DR2 2080246282881778048,1,<a refstr=ROWE_ET_AL__2014 href=https://ui.ads...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,0.023,-0.023,13.916&plusmn;0.023,13.448,0.026,-0.026,13.448&plusmn;0.026,13.372,NaN,NaN
3,Kepler-216 b,b,Kepler-216,NaN,NaN,TIC 270698499,Gaia DR2 2128067720065692672,1,<a refstr=ROWE_ET_AL__2014 href=https://ui.ads...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,0.021,-0.021,12.956&plusmn;0.021,12.695,0.022,-0.022,12.695&plusmn;0.022,12.621,0.021,-0.021
4,Kepler-32 c,c,Kepler-32,NaN,NaN,TIC 273590427,Gaia DR2 2080287892525359872,1,<a refstr=FABRYCKY_ET_AL__2012 href=https://ui...,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,...,0.023,-0.023,13.616&plusmn;0.023,12.901,0.024,-0.024,12.901&plusmn;0.024,12.757,0.024,-0.024


## 4. Star Lookup

Normalize user input to find any spacing/casing variant.


In [45]:
def find_star(name):
    norm = name.replace(' ','').upper()
    df_norm = planets_df['hostname'].str.replace(' ','').str.upper()
    mask = df_norm==norm
    if not mask.any():
        raise LookupError(f"Star '{name}' not found.")
    return planets_df[mask]

In [46]:
# Example
star_name = 'HD189733'
star_df = find_star(star_name)
star_df[['hostname','st_teff','st_mass','st_rad']].drop_duplicates()


,hostname,st_teff,st_mass,st_rad
462,HD 189733,5052.0,0.79,0.75


## 5. Compute Orbits & RV

Extract parameters, build time grid, compute orbits and RV signals.


In [47]:
# Parameters
periods = star_df['pl_orbper'].values
eccs    = star_df['pl_orbeccen'].fillna(0).values
Ks      = star_df['pl_rvamp'].fillna(0).values
omegas  = np.deg2rad(star_df['pl_orblper'].fillna(0).values)
T0s     = star_df['pl_tranmid'].fillna(0).values
M_star  = star_df['st_mass'].iloc[0]

# Time grid
time = np.linspace(0, periods.max(), 200)

# Compute a and RV
a_list = [semi_major_axis(P,M_star) for P in periods]
rv_list = [radvelocity(time,P,K,e,om,T0)
           for P,K,e,om,T0 in zip(periods,Ks,eccs,omegas,T0s)]
rv_total = np.sum(rv_list,axis=0)


## 6. Animation

Generate frames and save as GIF.


In [48]:
frames=[]
for i,t0 in enumerate(time):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,4),
                               gridspec_kw={'width_ratios':[1,3]})
    fig.patch.set_facecolor('black')
    ax1.axis('off'); ax2.axis('off')
    ax1.scatter(0,0,s=150,c='yellow',marker='*')
    for P,a,e in zip(periods,a_list,eccs):
        x_e,y_e=ellipse(a,e)
        ax1.plot(x_e,y_e,c='cyan')
        th=2*np.pi*(t0%P)/P
        r=a*(1-e**2)/(1+e*np.cos(th))
        ax1.scatter(r*np.cos(th),r*np.sin(th),c='white')
    ax1.set_aspect('equal')
    ax2.plot(time,rv_total)
    ax2.scatter(t0,rv_total[i],c='white')
    mplcyberpunk.make_lines_glow(ax2)
    ax2.set_xlabel('Time (days)')
    ax2.set_ylabel('RV (m/s)')
    fig.canvas.draw()
    img=np.frombuffer(fig.canvas.tostring_rgb(),dtype='uint8')
    img=img.reshape(fig.canvas.get_width_height()[::-1]+(3,))
    frames.append(img)
    plt.close(fig)

gif_path='orbit_rv_animation.gif'
imageio.mimsave(gif_path,frames,fps=10)
print(f"Saved GIF as {gif_path}")


Saved GIF as orbit_rv_animation.gif
